## Allele filp QC YML generator
This module takes in a table of sumstat, with the columns: #chr, theme1, theme2, theme3 and each rows as 1 chr and the sumstat of corresponding chr and generate a list of yml to be used

In [ ]:
[global]
# List of path to the index of sumstat, each correspond to 1 recipe file documenting the path to the sumstat of each chromosome.
parameter: sumstat_list = paths
# List of names that corresponding to each of the studies
parameter: name = list
parameter: cwd = path
import pandas as pd
input_list = sumstat_list
sumstat_list = pd.read_csv(input_list[0],sep = "\t")
sumstat_list = sumstat_list.sort_values('#chr')
for x in range(1,len(input_list)):
    sumstat_list = sumstat_list.merge(pd.read_csv(input_list[x],sep = "\t"), on = "#chr")
sumstat_list.columns = ["#chr"] + name
sumstat_inv = sumstat_list.values.tolist()
name = "_".join(sumstat_list.columns.values[1:len(sumstat_list.columns.values)].tolist())
## Assuming all the input sumstat are using the same header
parameter: CHR = "chrom"
parameter: POS = "pos"
parameter: A0 = "ref" 
parameter: A1 = "alt"
parameter: SNP = "variant_id"
parameter: STAT = "beta"
parameter: SE = "se"
parameter: P = "pval"
parameter: KEEP_AMBIGUOUS = "True"
parameter: container = str

In [ ]:
[yml_generator]
input: for_each  = "sumstat_inv"
output: f'{wd:a}/{name}.{_sumstat_inv[0]}/{name}.{_sumstat_inv[0]}.yml'
bash: expand = "$[ ]", stderr = f'{_output}.stderr', stdout = f'{_output}.stdout' , container = container
        echo '
        INPUT:' > $[_output]

        for i in $[" ".join(_sumstat_inv[1:len(_sumstat_inv)])] ; do 
        echo "  
            - $i:
                ID: CHR,POS,A0,A1
                CHR: $[CHR]
                POS: $[POS]
                A0: $[A0]
                A1: $[A1]
                SNP: $[SNP]
                STAT: $[STAT]
                SE: $[SE]
                P: $[P]" >> $[_output];
        done 
        echo "
        TARGET:
            - $[_sumstat_inv[1]]:
                ID: CHR,POS,A0,A1
                CHR: $[CHR]
                POS: $[POS]
                A0: $[A0]
                A1: $[A1]
                SNP: $[SNP]
                STAT: $[STAT]
                SE: $[SE]
                P: $[P]
        OUTPUT: $[_output:d]" >> $[_output]

In [ ]:
[yml_list]
input: output_from("yml_generator"),group_by = "all"
output: f'{wd:a}/yml_list.txt', f'{wd:a}/qc_sumstat_list.txt'
import pandas as pd
yml_df = pd.DataFrame({"#chr" : pd.read_csv(sumstat_list_path,sep = "\t")["#chr"].values.tolist()  ,  "dir" : _input})
yml_df.to_csv(_output[0],sep = "\t",index = 0)
data_dir_tmp = pd.Series(_input)
data_dir = [f'{x:d}' for x in data_dir_tmp ]
theme = sumstat_list.columns.values.tolist()[1:]
for i in theme:
    sumstat_list = sumstat_list.assign(**{i : data_dir+ pd.Series([f'/{x.split("/")[-1]}' for x in  sumstat_list[i].values.tolist()])  } )
sumstat_list.to_csv(_output[1],sep = "\t", index = 0)

In [ ]:
## updating all the cells in sumstat_list